BLACKJACK TRAINING:-

RULES OF BLACKJACK:-

1. Blackjack is played with one or more standard decks of 52 playing cards.
2. The goal of the game is to beat the dealer's hand without exceeding a total of 21 points.
3. Each card in the deck has a point value: numbered cards 2-10 are worth their face value, face cards (Jack, Queen, King) are worth 10 points each, and Aces can be worth 1 or 11 points, depending on the player's choice.
4. The game begins with each player, including the dealer, receiving two cards. The players' cards are dealt face-up, while one of the dealer's cards is dealt face-up, and the other is face-down (hole card).
5. Players take turns deciding whether to "hit" (receive another card from the dealer) or "stand" (keep their current hand).
6. If the player's hand exceeds 21 points, they "bust" and lose the round.
7. After all players have completed their turns, the dealer reveals their hole card.
8. The dealer must follow a specific rule set known as "house rules." Typically, the dealer must hit until their hand reaches a total of 17 or more points. If the dealer's hand exceeds 21 points, they bust.
9. If the dealer has not busted, the dealer's hand is compared to each player's hand individually.
10. If the player's hand has a higher point total than the dealer's hand without exceeding 21, the player wins and receives a payout of 1:1 (even money).
11. If the player's hand is equal to the dealer's hand, it is a "push" or a tie, and the player's bet is returned.
12. If the player's hand is lower in point value than the dealer's hand or if the player busts, the player loses the bet.
13. If a player's initial two cards have a point total of 21 (an Ace and a 10-point card, also known as a "natural" or "blackjack"), they win immediately, unless the dealer also has a blackjack, resulting in a push.
14. Insurance: If the dealer's face-up card is an Ace, players have the option to take "insurance" against the dealer having a blackjack. This is an additional bet of up to half the original bet. If the dealer has a blackjack, the insurance bet pays 2:1.
15. Some variations of Blackjack may allow additional options such as "splitting" pairs, "doubling down" (doubling the bet and receiving one more card), or "surrendering" (giving up the hand and losing only half the bet).
16. It's important to note that different casinos or regions may have slightly different rules and variations of Blackjack, so it's always a good idea to familiarize yourself with the specific rules of the table you're playing at.

OpenAI Gymnasium BlackJack Documentation Link:-

https://gymnasium.farama.org/environments/toy_text/blackjack/

Solving BlackJack:-

Installing and Importing Libraries:-

In [4]:
!pip install gymnasium==0.27.0


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 879.1/879.1 kB 3.8 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.0.1
    Uninstalling typing_extensions-4.0.1:
      Successfully uninstalled typing_extensions-4.0.1

[notice] A new release of pip is available: 23.0 -> 23.1.2
[notice] To update, run: pip install --upgrade pip


In [ ]:
from collections import defaultdict
import matplotlib.pyplot as plt
from matplotlib.patches import Patch
import numpy as np
import seaborn as sns
from tqdm import tqdm #for progress bar to show loading
import gymnasium as gym

In [ ]:
env = gym.make('Blackjack-v1',sab=True,render_mode="rgb_array")

Observing Environment:-

In [ ]:
#Resetting Environment
done=False
observation,info = env.reset()

#observation = (16,9,False)

Note that the observation is a Tuple consisting of 3 Values:

1. Players current sum
2. Value of dealers face-up card
3. Boolean wether the player holds a usable Ace card